In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import os.path as osp
sys.path.insert(1, osp.abspath('../'))

## SequenceGraphDataset example

In [ ]:
from modules.torch_dataset.sequence_graph_dataset import SequenceGraphDataset

dataset = SequenceGraphDataset(sequence_path_prefix='datasets/AIC20', 
                               sequence_names=['S01', 'S03', 'S04'], 
                                annotations_filename='gt.txt')

data = dataset[0]     # Access the first processed graph

In [5]:
import pandas as pd
edge = pd.read_json("datasets/AIC20/logs/S03/sequence_graph_edges.json")
edge.head()

,src_node_id,dst_node_id,src_obj_id,dst_obj_id,edge_labels
0,0,14,252,252,1
1,0,15,252,242,0
2,0,16,252,256,0
3,0,17,252,246,0
4,0,18,252,247,0


In [37]:
edge[(edge.src_obj_id == edge.dst_obj_id)].head()

,src_node_id,dst_node_id,src_obj_id,dst_obj_id,edge_labels
0,0,14,252,252,1
17,0,31,252,252,1
31,0,45,252,252,1
48,0,62,252,252,1
67,1,15,242,242,1


## ObjectGraphDataset example

In [3]:
from modules.torch_dataset.object_graph_dataset import ObjectGraphDataset


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/hectormejia/opt/anaconda3/envs/mcmt_env/lib/python3.9/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/hectormejia/opt/anaconda3/envs/mcmt_env/lib/python3.9/site-packages/libpyg.so, 0x0006): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  Referenced from: <593A788E-3AC3-3D15-B6FD-54CA34BBA4D5> /Users/hectormejia/opt/anaconda3/envs/mcmt_env/lib/python3.9/site-packages/libpyg.so
  Reason: tried: '/usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python' (no such file), '/usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python' (no such file), '/Library/Frameworks/Python.framework/Versions/3.10/Python' (no such file), '/System/Library/Frameworks/Python.framework/Version

In [4]:
from models.reid.resnet import resnet101_ibn_a
model = resnet101_ibn_a(model_path="/Users/hectormejia/Documents/TFM/mcmt_gnn/models/reid/resnet101_ibn_a_2.pth", device="cpu")

Loading pretrained model from /Users/hectormejia/Documents/TFM/mcmt_gnn/models/reid/resnet101_ibn_a_2.pth


In [54]:
import torchvision.transforms.v2 as transforms
augmentation = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomErasing(scale=(0.02, 0.15)),
        transforms.RandomRotation(degrees=0.15),
        transforms.RandomPerspective(distortion_scale=0.25)
    ])

In [5]:
from modules.torch_dataset.object_graph_dataset import ObjectGraphDataset
from modules.data_processor.annotations_processor import AnnotationsProcessor

data_df = AnnotationsProcessor(sequence_path='/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20', annotations_filename='gt.txt').consolidate_annotations(['S02'])
dataset = ObjectGraphDataset(data_df, 
                             sequence_path_prefix="/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20", 
                             sequence_names=['S02'], 
                             annotations_filename='gt.txt', 
                             reid_model=model,
                             num_ids_per_graph=-1, 
                             embeddings_per_it=48, 
                             resized_img_shape=(224, 224), 
                             orignal_img_shape=(1920, 1080), 
                             augmentation=None)

In [6]:
graph, node_df, edge_df, sampled_df = dataset[0]

Sampling object ids
Computing embeddings


  0%|          | 0/437 [00:00<?, ?it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


100%|██████████| 437/437 [1:04:51<00:00,  8.90s/it]   


Generating nodes
Generating edges
Processed edges for camera c006. Shapes:  63075 63075 63075 63075
Processed edges for camera c007. Shapes:  42050 42050 42050 42050
Processed edges for camera c008. Shapes:  21025 21025 21025 21025
Processed edges for camera c009. Shapes:  0 0 0 0


/Users/hectormejia/Documents/TFM/mcmt_gnn/modules/torch_dataset/object_graph_dataset.py:233: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343686209/work/torch/csrc/utils/tensor_new.cpp:248.)
  edge_idx = torch.tensor(edge_idx, dtype=torch.int64, device=self.device)


Creating PyG graph


In [58]:
from models.mcmt.rgnn import MOTMPNet
import yaml
import torch 

with open("/Users/hectormejia/Documents/TFM/mcmt_gnn/config/training_rgcnn.yml", "r") as config_file:
    config = yaml.safe_load(config_file)
    gnn_arch = config["gnn_arch"]

model_state = torch.load("/Users/hectormejia/Documents/TFM/mcmt_gnn/results/checkpoints/checkpoint_100.pt")
model = MOTMPNet(gnn_arch)
model.load_state_dict(model_state['model'])


<All keys matched successfully>

In [59]:
from modules.inference.inference_module import InferenceModule, InferenceEngineAbstract
inf_module = InferenceModule(model, sampled_df, "/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20")

In [121]:
pred_df = inf_module.predict_tracks((graph, node_df, edge_df))

torch.Size([2, 126150])


/Users/hectormejia/Documents/TFM/mcmt_gnn/modules/inference/postprocessing.py:285: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return id_pred, torch.tensor(whole_edges_prediction)


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'int'

In [120]:
import pandas as pd
gt_df = pd.read_csv("/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20/ground_truth_S02.txt", sep=' ', names=['camera','id', 'frame', 'xmin', 'ymin', 'width', 'height', 'xworld', 'yworld'])
gt_df[gt_df.camera == 6]

,camera,id,frame,xmin,ymin,width,height,xworld,yworld
0,6,96,1,539,606,273,277,-1,-1
1,6,97,1,374,373,249,219,-1,-1
2,6,98,1,688,345,183,178,-1,-1
3,6,103,1,577,259,132,113,-1,-1
4,6,104,1,712,160,108,92,-1,-1
...,...,...,...,...,...,...,...,...,...
5076,6,202,2109,1756,327,160,114,-1,-1
5077,6,203,2109,1456,241,129,109,-1,-1
5078,6,163,2110,920,714,847,366,-1,-1
5079,6,202,2110,1734,314,184,130,-1,-1


In [118]:
data_df = AnnotationsProcessor(sequence_path='/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20', annotations_filename='gt.txt').consolidate_annotations(['S02'])
data_df[data_df.camera == 'c006']

,frame,id,xmin,ymin,width,height,lost,occluded,generated,label,xmax,ymax,camera,sequence_name
0,1,96,541,609,268,272,1,-1,-1,-1,809,881,c006,S02
1,1,97,375,373,246,219,1,-1,-1,-1,621,592,c006,S02
2,1,98,686,343,186,182,1,-1,-1,-1,872,525,c006,S02
3,1,103,573,254,140,123,1,-1,-1,-1,713,377,c006,S02
4,1,104,707,154,118,104,1,-1,-1,-1,825,258,c006,S02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5076,2109,163,1031,729,774,349,1,-1,-1,-1,1805,1078,c006,S02
5077,2109,203,1452,236,137,119,1,-1,-1,-1,1589,355,c006,S02
5078,2110,202,1732,310,187,138,1,-1,-1,-1,1919,448,c006,S02
5079,2110,163,948,721,790,358,1,-1,-1,-1,1738,1079,c006,S02


In [103]:
from modules.inference.postprocessing import fix_annotation_frames
#pred_df.camera = pred_df.camera.apply(lambda cam: int(cam.replace('c', '')))
#pred_df.head()

In [104]:
pred_df = inf_module.data_df

In [105]:
import motmetrics as mm

# Retrieving only the necessary columns
#gt_df = gt_df[['frame', 'id', 'xmin', 'ymin', 'width', 'height']]
#pred_df = pred_df[['frame', 'id', 'xmin', 'ymin', 'width', 'height']]

# Avoid colisions in the frame index
gt_df, pred_df = fix_annotation_frames(gt_df, pred_df)

gt_df = gt_df.rename(columns={'frame': 'FrameId',
                               'xmin': 'X', 
                               'ymin': 'Y', 
                               'width': 'Width', 
                               'height': 'Height',
                               'id': 'Id'})

pred_df = pred_df.rename(columns={'frame': 'FrameId',
                               'xmin': 'X', 
                               'ymin': 'Y', 
                               'width': 'Width', 
                               'height': 'Height',
                               'id': 'Id'})


In [106]:
gt_df = gt_df.set_index(['FrameId', 'Id'])
pred_df = pred_df.set_index(['FrameId', 'Id'])

In [110]:

# Compute metrics using Pymotmetrics
mh = mm.metrics.create()
accumulator = mm.utils.compare_to_groundtruth(gt_df, pred_df, 'iou', distth=0.7)
metrics=[*mm.metrics.motchallenge_metrics, *['num_frames','idfp','idfn','idtp']]
summary = mh.compute(accumulator, metrics=metrics, name='MultiCam')

In [112]:
summary

,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,...,num_fragmentations,mota,motp,num_transfer,num_ascend,num_migrate,num_frames,idfp,idfn,idtp
MultiCam,0.705141,1.0,0.54457,0.54457,1.0,145,8,129,8,0,...,2754,0.54457,0.075597,0,0,0,7435,0.0,9544.0,11412.0
